In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,30653
2,Huelva,Confirmados PDIA 14 días,1333
3,Huelva,Tasa PDIA 14 días,"259,7579749400783"
4,Huelva,Confirmados PDIA 7 días,640
5,Huelva,Total Confirmados,30833
6,Huelva,Curados,28241
7,Huelva,Fallecidos,377


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  30653.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  8058.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 723 personas en los últimos 7 días 

Un positivo PCR cada 361 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,30653.0,640.0,1333.0,513170.0,124.715007,259.757975,154.0
Huelva-Costa,17672.0,312.0,614.0,289548.0,107.754155,212.054651,76.0
Condado-Campiña,9936.0,279.0,605.0,156231.0,178.581716,387.247089,73.0
Huelva (capital),8058.0,199.0,398.0,143837.0,138.351050,276.702100,55.0
Moguer,1248.0,49.0,115.0,21867.0,224.081950,525.906617,12.0
Almonte,1759.0,63.0,125.0,24507.0,257.069409,510.058351,12.0
San Juan del Puerto,488.0,16.0,27.0,9411.0,170.013814,286.898310,8.0
Palma del Condado (La),1138.0,38.0,86.0,10801.0,351.819276,796.222572,8.0
Lepe,2121.0,16.0,24.0,27880.0,57.388809,86.083214,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villarrasa,166.0,10.0,18.0,2211.0,452.284034,814.111262,1.0
Palma del Condado (La),1138.0,38.0,86.0,10801.0,351.819276,796.222572,8.0
Cabezas Rubias,37.0,3.0,5.0,706.0,424.929178,708.215297,0.0
Manzanilla,98.0,6.0,12.0,2118.0,283.286119,566.572238,3.0
Bollullos Par del Condado,959.0,26.0,79.0,14387.0,180.718704,549.106833,6.0
Calañas,189.0,8.0,15.0,2768.0,289.017341,541.907514,0.0
Moguer,1248.0,49.0,115.0,21867.0,224.081950,525.906617,12.0
Almonte,1759.0,63.0,125.0,24507.0,257.069409,510.058351,12.0
Lucena del Puerto,248.0,7.0,14.0,3261.0,214.658080,429.316161,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villalba del Alcor,481.0,1.0,7.0,3366.0,29.708853,207.961973,0.0,0.142857
Cartaya,1319.0,8.0,28.0,20083.0,39.834686,139.421401,1.0,0.285714
Bollullos Par del Condado,959.0,26.0,79.0,14387.0,180.718704,549.106833,6.0,0.329114
Nerva,174.0,4.0,12.0,5169.0,77.384407,232.153221,0.0,0.333333
Valverde del Camino,542.0,20.0,53.0,12750.0,156.862745,415.686275,3.0,0.377358
Isla Cristina,2139.0,15.0,39.0,21393.0,70.116393,182.302622,5.0,0.384615
Trigueros,294.0,7.0,18.0,7862.0,89.035869,228.949377,3.0,0.388889
Aracena,518.0,2.0,5.0,8255.0,24.227741,60.569352,0.0,0.400000
San Bartolomé de la Torre,293.0,4.0,10.0,3761.0,106.354693,265.886732,2.0,0.400000
